# Linear Regression from scratch

In [ ]:
!pip3 install plotly

In [1]:
import re
from pathlib import Path
from typing import Union, List
from plotly import express as px
from plotly import graph_objects as go

In [2]:
# Ensure that we have a `data` directory we use to store downloaded data
!mkdir -p data
data_dir: Path = Path('data')

In [3]:
# Downloading the "Auto Insurance in Sweden" data set
!wget -nc -P data https://www.math.muni.cz/~kolacek/docs/frvs/M7222/data/AutoInsurSweden.txt

--2022-01-11 16:11:18--  https://www.math.muni.cz/~kolacek/docs/frvs/M7222/data/AutoInsurSweden.txt
Resolving www.math.muni.cz (www.math.muni.cz)... 147.251.81.50
Connecting to www.math.muni.cz (www.math.muni.cz)|147.251.81.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 866 [text/plain]
Saving to: ‘data/AutoInsurSweden.txt’

AutoInsurSweden.txt 100%[===================>]     866  --.-KB/s    in 0s      

2022-01-11 16:11:24 (59.0 MB/s) - ‘data/AutoInsurSweden.txt’ saved [866/866]



In [4]:
!head -n 10 data/AutoInsurSweden.txt

Auto Insurance in Sweden

In the following data
X = number of claims
Y = total payment for all the claims in thousands of Swedish Kronor
for geographical zones in Sweden
Reference: Swedish Committee on Analysis of Risk Premium in Motor Insurance
http://college.hmco.com/mathematics/brase/understandable_statistics/7e/students/datasets/
       slr/frames/frame.html



In [5]:
# Create the Python path pointing to the `AutoInsurSweden.txt` file
insurance_data_path: Path = data_dir / 'AutoInsurSweden.txt'

In [6]:
# Read the `AutoInsurSweden.txt` file, extract the `x` and `y` values via regex and store them into vectors
xs: List[float] = []
ys: List[float] = []

with open(insurance_data_path) as file:
    content: str = file.read()
    for x, y in re.findall(r'([\d,]+)\t([\d,]+)', content):
        xs.append(float(x.replace(',', '.')))
        ys.append(float(y.replace(',', '.')))

In [7]:
# A convenience function which creates a scatter plot with an optional line
def plot(xs: List[float], ys: List[float], ys_pred: Union[List[float], None] = None) -> None:
    fig = px.scatter(x=xs, y=ys, labels={'x': 'Number of claims', 'y': 'Total payment'})
    # If present, add the line
    if ys_pred:
        fig.add_trace(
            go.Scatter(
                x=xs, y=ys_pred, name='Guess'
            )
        )
    fig.show()

In [8]:
plot(xs, ys)

In [9]:
# The linear function which describes a line
# Our goal is to find `m` and `b` such that the line most accurately "describes" the insurance data points
def predict(m: float, b: float, x: float) -> float:
    return m * x + b

assert predict(m=0, b=0, x=3) == 0

In [10]:
# SSE (sum of squared estimate of errors), the function we use to calculate how "wrong" we are
# "How much do the actual y values (`ys`) differ from our predicted y values (`ys_pred`)?"
def sum_squared_error(ys: List[float], ys_pred: List[float]) -> float:
    assert len(ys) == len(ys_pred)
    return sum([(y - ys_pred) ** 2 for y, ys_pred in zip(ys, ys_pred)])

assert sum_squared_error([1, 2, 3], [4, 5, 6]) == 27

In [12]:
# Our initial guess as to what `m` and `b` might be
m: float = 0
b: float = 200

# Predicting the y values based on our initial guess for the line
ys_pred: List[float] = [predict(m, b, x) for x in xs]

# Visualize the result
plot(xs, ys, ys_pred)

In [13]:
print(f'Initial guess for "m": {m}')
print(f'Initial guess for "b": {b}')

# Calculate how "off" we are via SSE
loss: float = sum_squared_error(ys, ys_pred)
print(f'SSE: {loss}')

Initial guess for "m": 0
Initial guess for "b": 200
SSE: 1125865.2999999996


In [14]:
# Find the best fitting line through the data points via Gradient Descent
m: float = 0
b: float = 200

print(f'Starting with "m": {m}')
print(f'Starting with "b": {b}')

Starting with "m": 0
Starting with "b": 200


In [15]:
epochs: int = 10000
learning_rate: float = 0.00001

for epoch in range(epochs):
    # Calculate predictions for `y` values given the current `m` and `b`
    ys_pred: List[float] = [predict(m, b, x) for x in xs]

    # Calculate and print the error
    if epoch % 1000 == True:
        loss: float = sum_squared_error(ys, ys_pred)
        print(f'Epoch {epoch} --> loss: {loss}')

    # Calculate the gradient
    # Taking the (partial) derivative of SSE with respect to `m` results in `2 * x ((m * x + b) - y)`
    grad_m: float = sum([2 * (predict(m, b, x) - y) * x for x, y in zip(xs, ys)])
    # Taking the (partial) derivative of SSE with respect to `b` results in `2 ((m * x + b) - y)`
    grad_b: float = sum([2 * (predict(m, b, x) - y) for x, y in zip(xs, ys)])
    
    # Take a small step in the direction of greatest decrease
    m = m + (grad_m * -learning_rate)
    b = b + (grad_b * -learning_rate)

print(f'Best estimate for "m": {m}')
print(f'Best estimate for "b": {b}')

Epoch 1 --> loss: 1111304.0949169993
Epoch 1001 --> loss: 367095.40067246475
Epoch 2001 --> loss: 159429.33008833785
Epoch 3001 --> loss: 101348.4050032304
Epoch 4001 --> loss: 85104.08618286082
Epoch 5001 --> loss: 80560.80637884357
Epoch 6001 --> loss: 79290.12266438038
Epoch 7001 --> loss: 78934.73246790287
Epoch 8001 --> loss: 78835.33543438878
Epoch 9001 --> loss: 78807.53565158854
Best estimate for "m": 3.4071723383619705
Best estimate for "b": 20.302521479691976


In [16]:
plot(xs, ys, ys_pred)

## From Linear Regression to Multiple Regression

The slope-intercept form we've used so far can easily be updated to work with multiple $x$ values. Here's the linear equation we've used so far:

$$
y = mx + b
$$

Having multiple $x$ values means that we'll also have multiple $m$ values (one for each $x$). However we'll still only deal with `1` intercept:

$$
y =  m_1x_1 + ... + m_nx_n + b
$$

Calculating a prediction for $y$ is as simple as solving the above equation for any given vector of $x$ values, vector of $m$ values and any given $b$ value.

There's one little trick we can apply given that we're now mostly dealing with vectors rather than scalar numbers. To make the computation more efficient we can use the `dot-product` which carries out almost the exact same calculation we described above. There's just one problem. The `dot-product` can only be used in vector calculations, however $b$ isn't a vector. As it turns out we can simply prepend the $b$ value to the $m$ vector and prepend a `1` to the $x$ vector. Doing this little trick makes it possible to use the `dot-product` calculation while also taking the $b$ value into account. Here's what we'd end up with when doing just that:

$$
\vec{x} = \begin{pmatrix}1 \\
x_{1} \\
\dots \\
x_{n}
\end{pmatrix}
\vec{m} = \begin{pmatrix}b \\
m_{1} \\
\dots \\
m_{n}
\end{pmatrix}
$$

$$ 
y = \vec{x} \cdot \vec{m} = \sum_{i=1}^n x_i m_i = x_1 \times m_1 + \dots + x_n \times m_n 
$$

Another nice side-effect of doing this is that the `partial derivative` calculations for the error function will also be easier since our usage of the `dot-product` reduced the number of variables we have to take into account to just 2 vectors $x$ and $m$.

## Gradient Descent

Let's apply this rule and compute the partial derivatives of our Paraboloid function $x^2 + y^2$ which we call $f$:

The first partial derivative we calculate is the derivative of $f$ with respect to $x$, treating $y$ as a constant:

$$ \frac{\partial}{\partial x}(x^2 + y^2) = 2x $$

The second partial derivative calculation follows the same pattern, deriving $f$ with respect to $y$ while treating $x$ as a constant:

$$ \frac{\partial}{\partial y}(x^2 + y^2) = 2y $$

Note: Don't get confused by the notation here. While you'd use $\frac{d}{dx}$ for "normal" derivatives you simply use $\frac{\partial}{\partial x}$ for partial derivatives.

And that's it. With those partial derivatives we're now able to compute any gradient for any point $p$ sitting on the plotted surface of function $f$. Let's translate our findings into code:

In [ ]:
def compute_gradient(vec: List[float]) -> List[float]:
    assert len(vec) == 2
    x: float = vec[0]
    y: float = vec[1]
    return [2 * x, 2 * y]

Translating the prose into a mental image, we have a function $f$ which produces a surface in a 3 dimensional space. Given any point $p$ on that surface we can use the gradient (via its partial derivatives) to find a vector pointing into the direction of greatest increase.

That's great, but we're dealing with an optimization problem and for a lot of such applications it's usually desirable to find a global or local minimum. Right now, the gradient vector is pointing upwards to the direction of greatest increase. We need to turn that vector into the opposite direction so that it points to the direction of greatest decrease.

Linear Algebra taught us that doing that is as simple as multiplying the gradient vector by $-1$. Another neat Linear Algebra trick is to multiply a vector by a number other than $1$ to change its magnitude (= its length). If we're for example multiplying the gradient vector by $0.25$ we would get a vector which has length $\frac{1}{4}$ of its original length.

We finally have all the building blocks in place to put together the Gradient Descent algorithm which eventually finds the nearest local minimum for any given function.

The algorithm works as follows.

Given a function $f$ we want to run Gradient Descent on:

1. Get the starting position $p$ (which is represented as a vector) on $f$
2. Compute the gradient at point $p$
3. Multiply the gradient by a negative "step size" (usually a value smaller than $1$)
4. Compute the next position of $p$ on the surface by adding the rescaled gradient vector to the vector $p$
5. Repeat step 2 with the new $p$ until convergence

Let's turn that into code. First, we should define a `compute_step` function which takes as parameters the vector $p$ and a "step size" (we call it `learning_rate`), and computes the next position of $p$ according to the $f$ functions gradient:

In [ ]:
def compute_step(curr_pos: List[float], learning_rate: float) -> List[float]:
    grad: List[float] = compute_gradient(curr_pos)
    grad[0] *= -learning_rate
    grad[1] *= -learning_rate
    next_pos: List[float] = [0, 0]
    next_pos[0] = curr_pos[0] + grad[0]
    next_pos[1] = curr_pos[1] + grad[1]
    return next_pos

Next up we should define a random starting position $p$:

In [ ]:
start_pos: List[float]

# Ensure that we don't start at a minimum (0, 0 in our case)
while True:
    start_x: float = randint(xs_start, xs_stop)
    start_y: float = randint(ys_start, ys_stop)
    if start_x != 0 and start_y != 0:
        start_pos = [start_x, start_y]
        break

print(start_pos)
# [6, -3]

And finally we wrap our compute_step function into a loop to iteratively walk down the surface and eventually reach a local minimum:

In [ ]:
epochs: int = 5000
learning_rate: float = 0.001

best_pos: List[float] = start_pos

for i in range(0, epochs):
    next_pos: List[float] = compute_step(best_pos, learning_rate)
    if i % 500 == 0:
        print(f'Epoch {i}: {next_pos}')
    best_pos = next_pos

print(f'Best guess for a minimum: {best_pos}')

## Ridge penalty

$$
\text{minimise } \Bigg(SSE + \lambda \sum_{j = 1}^{p} \beta_{j}^{2}\Bigg)
$$

The size of this penalty, referred to as $L^{2}$ (or `Euclidean`) norm, can take on a wide range of values, which is controlled by the tuning parameter $\lambda$. When $\lambda = 0$ there is no effect and our objective function equals the normal `OLS` regression objective function of simply minimizing `SSE`. However, as $\lambda \to \infty$, the penalty becomes large and forces the coefficients toward zero (but not all the way). 

## Lasso penalty

The only difference is that we swap out the $L^{2}$ norm for an $L^{1}$ norm:

$$
\text{minimise } \Bigg(SSE + \lambda \sum_{j = 1}^{p} | \beta_{j} | \Bigg)
$$

Whereas the ridge penalty pushes variables to approximately but not equal to zero, the lasso penalty will actually push coefficients all the way to zero.

## Elastic nets

A generalization of the ridge and lasso penalties, called the elastic net combines the two penalties:

$$
\text{minimise } \Bigg(SSE + \lambda_{1} \sum_{j = 1}^{p} \beta_{j}^{2} + \lambda_{2} \sum_{j = 1}^{p} | \beta_{j} | \Bigg)
$$

In [ ]:
#\big( \Big( \bigg( \Bigg(	
#\big) \Big) \bigg) \Bigg)	